# 数据清洗、信息提取

## 初始化

In [1]:
import pandas as pd
import time

## 制作消费数据集

In [2]:
amount = pd.read_csv('data2.csv',encoding='gb2312')
# 删除无效字段
del amount['TermSerNo']
del amount['conOperNo']
del amount['Index']
del amount['PeoNo']
# 筛选出消费记录
amount = amount[amount.Type=='消费']
del amount['FundMoney']
del amount['Type']
# 处理日期和时间字符串
amount['Time'] = amount['Date'].apply(lambda date: time.strftime('%H:%M',(time.strptime(date,'%Y/%m/%d %H:%M'))))
amount['Date'] = amount['Date'].apply(lambda date: date.split(' ')[0])
# 保存成新数据集
amount.to_csv('consume.csv')

## 绑定消费数据和学生信息

由于只有18级的学生信息，所以准确来说是结果只包含18级学生的消费数据。

In [3]:
student = pd.read_csv('data1.csv',encoding='gb2312')
# 删除合并后会失去意义的字段
del student['Index']
# 合并data1和data2数据集
data = pd.merge(student, amount, on=['CardNo'])
# 保存成新数据集
data.to_csv('data_grade18.csv')
print(data)

CardNo Sex   Major  AccessCardNo       Date  Money  Surplus  \
0       180001   男  18国际金融      19762330  2019/4/21    7.0     28.4   
1       180001   男  18国际金融      19762330  2019/4/22    3.5     24.9   
2       180001   男  18国际金融      19762330  2019/4/10   11.0      2.7   
3       180001   男  18国际金融      19762330  2019/4/15    7.0     43.2   
4       180001   男  18国际金融      19762330  2019/4/15    2.5     50.2   
...        ...  ..     ...           ...        ...    ...      ...   
232985  184339   女  18工业工程      11513762  2019/4/18    2.5     78.8   
232986  184339   女  18工业工程      11513762  2019/4/15    4.1     46.5   
232987  184339   女  18工业工程      11513762  2019/4/15    0.6     45.9   
232988  184339   女  18工业工程      11513762  2019/4/15    6.0     39.9   
232989  184339   女  18工业工程      11513762  2019/4/12    0.5     58.3   

        CardCount  TermNo  OperNo  Dept   Time  
0             206      41     249  第四食堂  18:30  
1             207      19     236  第一食堂  09:40  
2       